In [4]:
import mircxpol as mp

In [5]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import Angle, SkyCoord, EarthLocation, AltAz
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider
from IPython.display import display, HTML
import pandas as pd 

In [6]:
path_csv = "2022Oct19_W1S1.csv"
time = np.arange(4.5, 12.01, 0.01) * u.hour
upsand = SkyCoord.from_name("ups and")
ptime = Time("2022-10-19 0:00:00") + time 

df = pd.read_csv(path_csv)
MJD_data = df['MJD'].values
MJD_data = MJD_data.reshape(-1, 1)
HA_data = df['HA'].values
HA_data = HA_data.reshape(-1, 1)
VisRatio_data = df['VisRatio'].values
VisRatio_data = VisRatio_data.reshape(-1, 1)
PD_data = df['PhaseDiff'].values
PD_data = PD_data.reshape(-1, 1)
HA = Angle(time).value

chara = EarthLocation.of_site("CHARA")
upsandaltaz = upsand.transform_to(AltAz(obstime=ptime, location=chara))
alt = upsandaltaz.alt
az = upsandaltaz.az
# Find the index of the maximum altitude
zenith_idx = np.argmax(alt)
zenith_time = HA[zenith_idx]
HA = HA - zenith_time

In [7]:
def plot_func(r_m31, c_m31, r_m41, c_m41, r_m81, c_m81, r_m32, c_m32, r_m42, c_m42, r_m82, c_m82, I12, Q12, U12, V12, Qii, Uii, Vii):

    C11 = mp.C(1, Qii, Uii, Vii)
    C22 = mp.C(1, Qii, Uii, Vii)
    C12 = mp.C(I12, Q12, U12, V12)

    V_amplitude = np.zeros((len(time), 3))
    V_phase = np.zeros((len(time), 3))
    VR = np.zeros((len(time), 3))
    DP = np.zeros((len(time), 3))

    for i in range(len(time)):
        alt_i = alt[i].value
        az_i = az[i].value

        J1_i = mp.Jall(alt_i, az_i, r_m31, c_m31, r_m41, c_m41, r_m81, c_m81)
        J2_i = mp.Jall(alt_i, az_i, r_m32, c_m32, r_m42, c_m42, r_m82, c_m82)

        V_amplitude[i][0] = abs(mp.Vis(J1_i, J2_i, C12, C11, C22, 0)[0]) #Vis_total
        V_amplitude[i][1] = abs(mp.Vis(J1_i, J2_i, C12, C11, C22, 1)[0]) #Vis_HH
        V_amplitude[i][2] = abs(mp.Vis(J1_i, J2_i, C12, C11, C22, 2)[0]) #Vis_VV

        V_phase[i][0] = mp.Vis(J1_i, J2_i, C12, C11, C22, 0)[1]
        V_phase[i][1] = mp.Vis(J1_i, J2_i, C12, C11, C22, 1)[1]
        V_phase[i][2] = mp.Vis(J1_i, J2_i, C12, C11, C22, 2)[1]

        VR[i][0] = abs(mp.Vis_diff(J1_i, J2_i, C12, C11, C22)[0]) #VH
        VR[i][1] = abs(mp.Vis_diff(J1_i, J2_i, C12, C11, C22)[1]) #VV
        VR[i][2] = abs(mp.Vis_diff(J1_i, J2_i, C12, C11, C22)[1])/ abs(mp.Vis_diff(J1_i, J2_i, C12, C11, C22)[0])  #VV/VH

        DP[i][0] = mp.Vis_diff(J1_i, J2_i, C12, C11, C22)[2]#PH
        DP[i][1] = mp.Vis_diff(J1_i, J2_i, C12, C11, C22)[3] #PV
        DP[i][2] = mp.Vis_diff(J1_i, J2_i, C12, C11, C22)[3] - mp.Vis_diff(J1_i, J2_i, C12, C11, C22)[2]


        V_total = V_amplitude[:, 0]
        V_H = V_amplitude[:, 1]
        V_V = V_amplitude[:, 2]

        V_total_phase = V_phase[:, 0]
        V_H_phase = V_phase[:, 1] 
        V_V_phase = V_phase[:, 2] 

        V_H1 = VR[:, 0]
        V_V1 = VR[:, 1]
        V_Ratio = VR[:, 2]

        V_H_phase1 = DP[:, 0] 
        V_V_phase1 = DP[:, 1]
        D_phase = DP[:, 2]
        # return HA, V_total, V_H, V_V, V_total_phase, V_H_phase, V_V_phase, V_Ratio, D_phase, zenith_idx, HA_data, VisRatio_data, PD_data


    plt.figure(figsize=(20, 12))
    plt.subplot(2, 2, 3)
    plt.axvline(x = HA[zenith_idx], color='green', linestyle='--', label='zenith')
    plt.plot(HA, V_total, label="Total", linewidth=3, color='black', linestyle='--')
    plt.plot(HA, V_H, label="Horizontal")
    plt.plot(HA, V_V, label="Vertical")
    # plt.title(sys.argv[2] + ', ' + sys.argv[3], fontsize = 15)
    plt.ylabel("Visibility", fontsize = 15)
    plt.xlabel("Hour Angle", fontsize = 15)
    plt.grid()
    plt.legend(fontsize = 15)

    plt.subplot(2, 2, 4)
    plt.axvline(x = HA[zenith_idx], color='green', linestyle='--', label='zenith')
    plt.plot(HA, V_total_phase, label="Total", linewidth=3, color='black', linestyle='--')
    plt.plot(HA, V_H_phase, label="Horizontal")
    plt.plot(HA, V_V_phase, label="Vertical")
    plt.ylabel(r'$Phase (degree)$', fontsize = 15)
    plt.xlabel("Hour Angle", fontsize = 15)
    plt.grid()
    plt.legend(fontsize = 15)


    plt.subplot(2, 2, 1)
    plt.axvline(x = HA[zenith_idx], color='green', linestyle='--', label='zenith')
    plt.plot(HA, V_Ratio, label="Ratio", linewidth=3, color='black', linestyle='--')

    plt.scatter(HA_data, VisRatio_data)#, label="W1-S1")
    # plt.title(sys.argv[2] + ', ' + sys.argv[3], fontsize = 15)
    plt.ylabel("Visibility Ratio", fontsize = 15)
    plt.xlabel("Hour Angle", fontsize = 15)
    plt.grid()
    plt.legend(fontsize = 15)

    plt.subplot(2, 2, 2)
    plt.plot(HA, D_phase, label="Diff", linewidth=3, color='black', linestyle='--')
    plt.scatter(HA_data, PD_data)#, label="W1-S1")
    plt.axvline(x = HA[zenith_idx], color='green', linestyle='--', label='zenith')
    plt.ylabel(r'$Differential~phase~(degrees)$', fontsize = 15)
    plt.xlabel("Hour Angle", fontsize = 15)
    plt.grid()
    plt.legend(fontsize = 15)

    plt.tight_layout()
    plt.show()
    


In [8]:
def main():

    r_m31_widget = FloatSlider(min=0.100, max=1.000, step=0.001, value=1.000, description="M1-3", readout_format='.3f')
    c_m31_widget = FloatSlider(min=0.000, max=2.000, step=0.001, value=0.000, description="Phase M1-3", readout_format='.3f')
    r_m41_widget = FloatSlider(min=0.100, max=1.000, step=0.001, value=1.000, description = "M4-7", readout_format='.3f')
    c_m41_widget = FloatSlider(min=0.000, max=2.000, step=0.001, value=0.000, description = "Phase M4-7", readout_format='.3f')
    r_m81_widget = FloatSlider(min=0.100, max=1.000, step=0.001, value=1.000, description = "M8-19", readout_format='.3f')
    c_m81_widget = FloatSlider(min=0.000, max=2.000, step=0.001, value=0.000, description = "Phase M8-19", readout_format='.3f')

    r_m32_widget = FloatSlider(min=0.100, max=1.000, step=0.001, value=1.000, description = "M1-3", readout_format='.3f')
    c_m32_widget = FloatSlider(min=0.000, max=2.000, step=0.001, value=0.000, description = "Phase M1-3", readout_format='.3f')
    r_m42_widget = FloatSlider(min=0.100, max=1.000, step=0.001, value=1.000, description = "M4-7", readout_format='.3f')
    c_m42_widget = FloatSlider(min=0.000, max=2.000, step=0.001, value=0.000, description = "Phase M4-7", readout_format='.3f')
    r_m82_widget = FloatSlider(min=0.100, max=1.000, step=0.001, value=1.000, description = "M8-19", readout_format='.3f')
    c_m82_widget = FloatSlider(min=0.000, max=2.000, step=0.001, value=0.000, description = "Phase M8-19", readout_format='.3f')

    I12_widget = FloatSlider(min=0.000, max=1.000, step=0.001, value=1.00, description = "I12", readout_format='.3f')
    Q12_widget = FloatSlider(min=-1.000, max=1.000, step=0.001, value=0.00, description = "Q12", readout_format='.3f')
    U12_widget = FloatSlider(min=-1.000, max=1.000, step=0.001, value=0.00, description = "U12", readout_format='.3f')
    V12_widget = FloatSlider(min=-1.000, max=1.000, step=0.001, value=0.00, description = "V12", readout_format='.3f')

    # Iii_widget = FloatSlider(min=0.000, max=1.000, step=0.001, value=1.00, description = "Iii", readout_format='.3f')
    Qii_widget = FloatSlider(min=-1.000, max=1.000, step=0.001, value=0.00, description = "Qii", readout_format='.3f')
    Uii_widget = FloatSlider(min=-1.000, max=1.000, step=0.001, value=0.00, description = "Uii", readout_format='.3f')
    Vii_widget = FloatSlider(min=-1.000, max=1.000, step=0.001, value=0.00, description = "Vii", readout_format='.3f')

    left_title = widgets.HTML(value="<div style='text-align: center;'><h3>beam1</h3></div>")
    center_title = widgets.HTML(value="<div style='text-align: center;'><h3>beam2</h3></div>")
    right_title = widgets.HTML(value="<div style='text-align: center;'><h3>Stokes parameters</h3></div>")

    left_box = widgets.VBox([left_title, r_m31_widget, c_m31_widget, r_m41_widget, c_m41_widget, r_m81_widget, c_m81_widget])
    center_box = widgets.VBox([center_title, r_m32_widget, c_m32_widget, r_m42_widget, c_m42_widget, r_m82_widget, c_m82_widget])
    right_box = widgets.VBox([right_title, I12_widget, Q12_widget, U12_widget, V12_widget, Qii_widget, Uii_widget, Vii_widget]) # Iii_widget,
    all_widgets = widgets.HBox([left_box, center_box, right_box])

    out = widgets.interactive_output(plot_func, {'r_m31': r_m31_widget, 'c_m31': c_m31_widget, 'r_m41': r_m41_widget, 
                                                'c_m41': c_m41_widget, 'r_m81': r_m81_widget, 'c_m81': c_m81_widget, 
                                                'r_m32': r_m32_widget, 'c_m32': c_m32_widget, 'r_m42': r_m42_widget, 
                                                'c_m42': c_m42_widget, 'r_m82': r_m82_widget, 'c_m82': c_m82_widget,
                                                'I12': I12_widget, 'Q12': Q12_widget, 'U12': U12_widget, 'V12': V12_widget, 
                                                'Qii': Qii_widget, 'Uii': Uii_widget, 'Vii': Vii_widget})  # 'Iii': Iii_widget


    if I12_widget.value +Q12_widget.value > 1 - Qii_widget.value:
        print(f"I12 + Q12 is not within the expected range")

    elif I12_widget.value - Q12_widget.value > 1 - Qii_widget.value:
        print(f"I12 - Q12 is not within the expected range")

    css = """
    .bg-red {
        background-color: rgba(255, 0, 0, 0.25) !important; /* Adjust the last value (0.5) to change the transparency */
    }

    """

    # Create an HTML widget to apply CSS
    html = widgets.HTML(value="<style>{}</style>".format(css))

    # Function to update widget background color based on condition
    def update_color(change):
        if I12_widget.value +Q12_widget.value > 1 - Qii_widget.value:
            all_widgets.add_class("bg-red")
        elif I12_widget.value - Q12_widget.value > 1 - Qii_widget.value:
            all_widgets.add_class("bg-red")
        else:
            all_widgets.remove_class("bg-red")

    # Link widget value change to color update function
    I12_widget.observe(update_color, names='value')
    Q12_widget.observe(update_color, names='value')
    Qii_widget.observe(update_color, names='value')

    display(html)
    display(all_widgets, out)


if __name__ == "__main__":
    main()

HTML(value='<style>\n    .bg-red {\n        background-color: rgba(255, 0, 0, 0.25) !important; /* Adjust the …

Output()